# **Setores Industriais Canadá**

## *- Importando Bibliotecas*

In [1]:
import re
import pandas as pd
from io import StringIO

## *- Extraindo Dados*

In [2]:
with open("Dados/Landing/1410035501-noSymbol.csv", "r") as file:
    conteudo = file.read()

    texto_csv = ""

    conteudo_linhas = conteudo.split("\n")
    conteudo_linhas = conteudo_linhas[11:32]
    del conteudo_linhas[1]

    colunas = conteudo_linhas[0]

    lista_meses = colunas.split(",")
    meses = len(lista_meses[1:])

    for i, linha in enumerate(conteudo_linhas):

        valores = linha.split('"')

        valores = valores[1::2]

        valor_final = ";".join(valores)

        texto_csv += f"{valor_final}\n"


# *- Transformando Dados*

In [ ]:
arquivo_csv = StringIO(texto_csv)

df = pd.read_csv(arquivo_csv, sep=";")

display(df)

In [ ]:
print(df.columns)

In [5]:
#Renomeando a primeira coluna
df = df.rename(columns={"North American Industry Classification System (NAICS) 6": "North American Industry Classification System (NAICS)"})

In [ ]:
first = df.columns[0]

df[first] = df[first].apply(lambda x: re.sub(r"\d", "", x))

display(df[first])

In [ ]:
display(df.head())

In [ ]:


df.iloc[:,1:] = df.iloc[:,1:].map(lambda cel: re.sub(",","", cel))
df.iloc[:,1:] = df.iloc[:,1:].astype(float)



display(df)

In [ ]:
# Traduzindo as colunas
column_translation_dict = {
    'North American Industry Classification System (NAICS)': 'Sistema de Classificação de Indústrias da América do Norte (SCIAN)',
    'January': 'Janeiro',
    'February': 'Fevereiro',
    'March': 'Março',
    'April': 'Abril',
    'May': 'Maio',
    'June': 'Junho',
    'July': 'Julho',
    'August': 'Agosto',
    'September': 'Setembro',
    'October': 'Outubro',
    'November': 'Novembro',
    'December': 'Dezembro'
}


def traduzir_coluna(column):
    for eng, ptbr in column_translation_dict.items():
        if eng in column:
            column = column.replace(eng, ptbr)
    return column


df.columns = df.columns.map(lambda column: column.strip())
df.columns = df.columns.map(traduzir_coluna)

display(df)

In [ ]:
# Traduzindo a primeira coluna
first = df.columns[0]

cell_translation_dict = {
    "Total employed, all industries": "Total empregado, todas as indústrias",
    "Goods-producing sector": "Setor de produção de bens",
    "Agriculture": "Agricultura",
    "Forestry, fishing, mining, quarrying, oil and gas": "Silvicultura, pesca, mineração, extração de pedreiras, petróleo e gás",
    "Utilities": "Serviços públicos",
    "Construction": "Construção",
    "Manufacturing": "Manufatura",
    "Services-producing sector": "Setor de produção de serviços",
    "Wholesale and retail trade": "Comércio atacadista e varejista",
    "Transportation and warehousing": "Transporte e armazenamento",
    "Finance, insurance, real estate, rental and leasing": "Finanças, seguros, imóveis, aluguel e leasing",
    "Professional, scientific and technical services": "Serviços profissionais, científicos e técnicos",
    "Business, building and other support services": "Serviços de negócios, construção e outros serviços de apoio",
    "Educational services": "Serviços educacionais",
    "Health care and social assistance": "Cuidados com a saúde e assistência social",
    "Information, culture and recreation": "Informação, cultura e recreação",
    "Accommodation and food services": "Acomodações e serviços de alimentação",
    "Other services (except public administration)": "Outros serviços (exceto administração pública)",
    "Public administration": "Administração pública"
}


def traduzir_cell(cell):
    for eng, ptbr in cell_translation_dict.items():
        if eng in cell:
            cell = cell.replace(eng,ptbr)
    return cell


df[first] = df[first].map(lambda cell: cell.strip())
df[first] = df[first].map(traduzir_cell)

display(df)

In [11]:
df.to_csv("Dados/Tratados/DadosSetoresCanada.csv", sep=";", index=False)

# *- Loading na S3*

In [12]:
import os
import boto3
from dotenv import load_dotenv


# Carregar variáveis
load_dotenv(r"credentials/.env")

# Configure as cresdencias da AWS
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

# Crie um client S3
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

# Carregue o arquivo para o S3
s3.upload_file(
    Filename='Dados/Tratados/DadosSetoresCanada.csv',
    Bucket='projetofinal-consuelo',
    Key='DadosSetoresCanada.csv'
)

# **Setores e Subsetores NAICS (2020-2023)**

## *- Importando Bibliotecas*

In [2]:
from dotenv import load_dotenv
import pandas as pd
import boto3
import wget
import ssl
import os

## *- Extração*

In [3]:
# Criando diretório (caso não exista)

try:
    os.mkdir("Dados")
    os.mkdir("Dados/Landing")
except:
    pass

In [4]:
ssl._create_default_https_context = ssl._create_stdlib_context

url = 'https://data.ca.gov/dataset/3f08b68e-1d1a-4ba4-a07d-1ec3392ed191/resource/ca165fad-4f16-48c7-808e-8b5222bc4182/download/qcew-2020-2022.csv'
wget.download(url, 'Dados/Landing/qcew2020-2022.csv')

url = 'https://data.ca.gov/dataset/3f08b68e-1d1a-4ba4-a07d-1ec3392ed191/resource/119eef38-3b59-499f-8f7c-9bea4768469d/download/qcew-2023.csv'
wget.download(url, "Dados/Landing/qcew2023.csv")

'Dados/Landing/qcew2023 (1).csv'

## *- Transformação*

### -- Carregamento dos Dados

In [5]:
df2022 = pd.read_csv("Dados/Landing/qcew2020-2022.csv")
df2023 = pd.read_csv("Dados/Landing/qcew2023.csv")

### *-- Traduzindo Colunas*

In [6]:
colunas_traduzidas = {
    'Area Type': 'Tipo de Área',
    'Area Name': 'Nome da Área',
    'Year': 'Ano',
    'Time Period': 'Período de Tempo',
    'Ownership': 'Propriedade',
    'NAICS Level': 'Nível NAICS',
    'NAICS Code': 'Código NAICS',
    'Industry Name': 'Nome da Indústria',
    'Establishments': 'Estabelecimentos',
    'Average Monthly Employment': 'Emprego Médio Mensal',
    '1st Month Emp': 'Emprego no 1º Mês',
    '2nd Month Emp': 'Emprego no 2º Mês',
    '3rd Month Emp': 'Emprego no 3º Mês',
    'Total Wages (All Workers)': 'Salários Totais (Todos os Trabalhadores)',
    'Average Weekly Wages': 'Salário Médio Semanal'
}


df2022.rename(columns=colunas_traduzidas, inplace = True)
df2023.rename(columns=colunas_traduzidas, inplace = True)

In [7]:
display(df2022.head())
print("\n-------------------\n")
display(df2023.head())

,Tipo de Área,Nome da Área,Ano,Período de Tempo,Propriedade,Nível NAICS,Código NAICS,Nome da Indústria,Estabelecimentos,Emprego Médio Mensal,Emprego no 1º Mês,Emprego no 2º Mês,Emprego no 3º Mês,Salários Totais (Todos os Trabalhadores),Salário Médio Semanal
0,County,Alameda County,2020,1st Qtr,Federal Government,2,1024,Professional and Business Services,2,9,9,9,9,279730.0,2391
1,County,Alameda County,2020,1st Qtr,Federal Government,3,491,Postal Service,28,3189,3198,3181,3189,54692130.0,1319
2,County,Alameda County,2020,1st Qtr,Federal Government,3,541,Professional and Technical Services,2,9,9,9,9,279730.0,2391
3,County,Alameda County,2020,1st Qtr,Federal Government,5,54133,Engineering Services,1,2,2,2,2,31831.0,1224
4,County,Alameda County,2020,1st Qtr,Federal Government,2,62,Health Care and Social Assistance,1,376,384,376,369,8804659.0,1800



-------------------



,Tipo de Área,Nome da Área,Ano,Período de Tempo,Propriedade,Nível NAICS,Código NAICS,Nome da Indústria,Estabelecimentos,Emprego Médio Mensal,Emprego no 1º Mês,Emprego no 2º Mês,Emprego no 3º Mês,Salários Totais (Todos os Trabalhadores),Salário Médio Semanal
0,County,Alameda County,2023,1st Qtr,Private,6.0,524298,All Other Insurance Related Activities,10,223,223,225,223,8.435605e+06,2901
1,County,Alameda County,2023,1st Qtr,Private,2.0,54,Professional and Technical Services,7315,76545,76542,76918,76176,3.445536e+09,3463
2,County,Alameda County,2023,1st Qtr,Private,6.0,541512,Computer Systems Design Services,814,5819,5878,5784,5795,2.162159e+08,2858
3,County,Alameda County,2023,1st Qtr,Private,6.0,541850,Display Advertising,11,160,157,162,163,5.615161e+06,2688
4,County,Alameda County,2023,1st Qtr,Private,6.0,551114,Managing Offices,211,16740,16717,16743,16761,9.291885e+08,4270


### *-- Padronizando Tipos*

In [8]:
#Primeiro transformando para inteiro para tirar o ponto flutuante
df2023['Nível NAICS'] = df2023['Nível NAICS'].astype(pd.Int64Dtype())
df2022['Nível NAICS'] = df2022['Nível NAICS'].astype(pd.Int64Dtype())

#Transformando para string
df2023['Nível NAICS'] = df2023['Nível NAICS'].astype(pd.StringDtype())
df2022['Código NAICS'] = df2022['Código NAICS'].astype(pd.StringDtype())


#Transformando Salário Médio em float
df2023['Salário Médio Semanal'] = df2023['Salário Médio Semanal'].astype(float)
df2022['Salário Médio Semanal'] = df2022['Salário Médio Semanal'].astype(float)

### *-- Juntando Tabelas*

In [9]:
df = pd.concat([df2022, df2023], ignore_index=True)

In [10]:
display(df)

,Tipo de Área,Nome da Área,Ano,Período de Tempo,Propriedade,Nível NAICS,Código NAICS,Nome da Indústria,Estabelecimentos,Emprego Médio Mensal,Emprego no 1º Mês,Emprego no 2º Mês,Emprego no 3º Mês,Salários Totais (Todos os Trabalhadores),Salário Médio Semanal
0,County,Alameda County,2020,1st Qtr,Federal Government,2,1024,Professional and Business Services,2,9,9,9,9,279730.0,2391.0
1,County,Alameda County,2020,1st Qtr,Federal Government,3,491,Postal Service,28,3189,3198,3181,3189,54692130.0,1319.0
2,County,Alameda County,2020,1st Qtr,Federal Government,3,541,Professional and Technical Services,2,9,9,9,9,279730.0,2391.0
3,County,Alameda County,2020,1st Qtr,Federal Government,5,54133,Engineering Services,1,2,2,2,2,31831.0,1224.0
4,County,Alameda County,2020,1st Qtr,Federal Government,2,62,Health Care and Social Assistance,1,376,384,376,369,8804659.0,1800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089532,County,Yuba County,2023,Annual,Private,<NA>,541219,Other Accounting Services,4,14,0,0,0,314380.0,442.0
1089533,County,Yuba County,2023,Annual,Private,<NA>,54143,Graphic Design Services,5,4,0,0,0,133497.0,604.0
1089534,County,Yuba County,2023,Annual,Private,<NA>,5416,Management & Technical Consulting Svc,12,79,0,0,0,7705299.0,1868.0
1089535,County,Yuba County,2023,Annual,Private,<NA>,54194,Veterinary Services,5,31,0,0,0,1386790.0,849.0


### *-- Separando Períodos Anuais dos Trimestrais*

In [11]:
df_anual = df[df['Período de Tempo'] == 'Annual']
df_trimestral = df[df['Período de Tempo'] != 'Annual']

df = df_trimestral # Backup código

In [12]:
display(df.head())
print("\n-------------------\n")
display(df_anual.head())

,Tipo de Área,Nome da Área,Ano,Período de Tempo,Propriedade,Nível NAICS,Código NAICS,Nome da Indústria,Estabelecimentos,Emprego Médio Mensal,Emprego no 1º Mês,Emprego no 2º Mês,Emprego no 3º Mês,Salários Totais (Todos os Trabalhadores),Salário Médio Semanal
0,County,Alameda County,2020,1st Qtr,Federal Government,2,1024,Professional and Business Services,2,9,9,9,9,279730.0,2391.0
1,County,Alameda County,2020,1st Qtr,Federal Government,3,491,Postal Service,28,3189,3198,3181,3189,54692130.0,1319.0
2,County,Alameda County,2020,1st Qtr,Federal Government,3,541,Professional and Technical Services,2,9,9,9,9,279730.0,2391.0
3,County,Alameda County,2020,1st Qtr,Federal Government,5,54133,Engineering Services,1,2,2,2,2,31831.0,1224.0
4,County,Alameda County,2020,1st Qtr,Federal Government,2,62,Health Care and Social Assistance,1,376,384,376,369,8804659.0,1800.0



-------------------



,Tipo de Área,Nome da Área,Ano,Período de Tempo,Propriedade,Nível NAICS,Código NAICS,Nome da Indústria,Estabelecimentos,Emprego Médio Mensal,Emprego no 1º Mês,Emprego no 2º Mês,Emprego no 3º Mês,Salários Totais (Todos os Trabalhadores),Salário Médio Semanal
202638,County,Alpine County,2020,Annual,Private,2,44-45,Retail Trade,4,13,0,0,0,315141.0,485.0
202639,County,Butte County,2020,Annual,Private,6,445210,Meat Markets,4,25,0,0,0,1210395.0,940.0
202640,County,Calaveras County,2020,Annual,Private,3,221,Utilities,5,108,0,0,0,15775439.0,2820.0
202641,County,Calaveras County,2020,Annual,Private,2,23,Construction,180,866,0,0,0,51419983.0,1142.0
202642,County,Alameda County,2020,Annual,Private,6,334510,Electromedical Apparatus Manufacturing,37,1692,0,0,0,285925551.0,3250.0


### *-- Traduzindo Valores*

In [13]:
# print(set(df["Período de Tempo"]))
# print(set(df["Tipo de Área"]))
# print(set(df["Nome da Área"]))
# print(set(df["Propriedade"]))


# Pedindo tradução ao GPT

temp_dict = {
    '1st Qtr': 'Trimestre 1',
    '2nd Qtr': 'Trimestre 2',
    '3rd Qtr': 'Trimestre 3',
    '4th Qtr': 'Trimestre 4',
    'Annual': 'Anual'
}


tipo_dict = {
    'United States': 'Estados Unidos',
    'California - Statewide': 'Califórnia - Estado',
    'County': 'Condado'
}

nome_dict = {
    'Marin County': 'Condado de Marin',
    'California': 'Califórnia',
    'Nevada County': 'Condado de Nevada',
    'Siskiyou County': 'Condado de Siskiyou',
    'Humboldt County': 'Condado de Humboldt',
    'Santa Barbara County': 'Condado de Santa Bárbara',
    'Fresno County': 'Condado de Fresno',
    'Kern County': 'Condado de Kern',
    'Shasta County': 'Condado de Shasta',
    'Sierra County': 'Condado de Sierra',
    'Merced County': 'Condado de Merced',
    'Lake County': 'Condado de Lake',
    'Contra Costa County': 'Condado de Contra Costa',
    'Solano County': 'Condado de Solano',
    'Santa Clara County': 'Condado de Santa Clara',
    'Butte County': 'Condado de Butte',
    'Ventura County': 'Condado de Ventura',
    'Sacramento County': 'Condado de Sacramento',
    'Mono County': 'Condado de Mono',
    'Orange County': 'Condado de Orange',
    'Calaveras County': 'Condado de Calaveras',
    'Del Norte County': 'Condado de Del Norte',
    'San Benito County': 'Condado de San Benito',
    'Trinity County': 'Condado de Trinity',
    'United States': 'Estados Unidos',
    'Kings County': 'Condado de Kings',
    'Stanislaus County': 'Condado de Stanislaus',
    'Plumas County': 'Condado de Plumas',
    'Mariposa County': 'Condado de Mariposa',
    'Inyo County': 'Condado de Inyo',
    'Riverside County': 'Condado de Riverside',
    'San Diego County': 'Condado de San Diego',
    'Colusa County': 'Condado de Colusa',
    'San Luis Obispo County': 'Condado de San Luis Obispo',
    'San Bernardino County': 'Condado de San Bernardino',
    'Yuba County': 'Condado de Yuba',
    'Napa County': 'Condado de Napa',
    'El Dorado County': 'Condado de El Dorado',
    'Glenn County': 'Condado de Glenn',
    'Monterey County': 'Condado de Monterey',
    'Santa Cruz County': 'Condado de Santa Cruz',
    'Yolo County': 'Condado de Yolo',
    'Amador County': 'Condado de Amador',
    'Tuolumne County': 'Condado de Tuolumne',
    'Lassen County': 'Condado de Lassen',
    'Modoc County': 'Condado de Modoc',
    'Los Angeles County': 'Condado de Los Angeles',
    'San Joaquin County': 'Condado de San Joaquin',
    'Alpine County': 'Condado de Alpine',
    'San Francisco County': 'Condado de San Francisco',
    'Alameda County': 'Condado de Alameda',
    'Tehama County': 'Condado de Tehama',
    'Tulare County': 'Condado de Tulare',
    'Sutter County': 'Condado de Sutter',
    'Imperial County': 'Condado de Imperial',
    'Placer County': 'Condado de Placer',
    'San Mateo County': 'Condado de San Mateo',
    'Mendocino County': 'Condado de Mendocino',
    'Madera County': 'Condado de Madera',
    'Sonoma County': 'Condado de Sonoma'
}


prop_dict = {
    'Total Government': 'Governo Total',
    'Total U.I. Covered': 'Total Coberto pelo Seguro Desemprego',
    'Federal Government': 'Governo Federal',
    'Local Government': 'Governo Local',
    'Total Covered': 'Total Coberto',
    'Private': 'Setor Privado',
    'State Government': 'Governo Estadual'
}

In [14]:
# Substituindo valores

df["Período de Tempo"] = df["Período de Tempo"].map(temp_dict)
df["Tipo de Área"] = df["Tipo de Área"].map(tipo_dict)
df["Nome da Área"] = df["Nome da Área"].map(nome_dict)
df["Propriedade"] = df["Propriedade"].map(prop_dict)

df_anual["Período de Tempo"] = df_anual["Período de Tempo"].map(temp_dict)
df_anual["Tipo de Área"] = df_anual["Tipo de Área"].map(tipo_dict)
df_anual["Nome da Área"] = df_anual["Nome da Área"].map(nome_dict)
df_anual["Propriedade"] = df_anual["Propriedade"].map(prop_dict)

In [15]:
display(df.head())
print("\n-------------------\n")
display(df_anual.head())

,Tipo de Área,Nome da Área,Ano,Período de Tempo,Propriedade,Nível NAICS,Código NAICS,Nome da Indústria,Estabelecimentos,Emprego Médio Mensal,Emprego no 1º Mês,Emprego no 2º Mês,Emprego no 3º Mês,Salários Totais (Todos os Trabalhadores),Salário Médio Semanal
0,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,2,1024,Professional and Business Services,2,9,9,9,9,279730.0,2391.0
1,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,3,491,Postal Service,28,3189,3198,3181,3189,54692130.0,1319.0
2,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,3,541,Professional and Technical Services,2,9,9,9,9,279730.0,2391.0
3,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,5,54133,Engineering Services,1,2,2,2,2,31831.0,1224.0
4,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,2,62,Health Care and Social Assistance,1,376,384,376,369,8804659.0,1800.0



-------------------



,Tipo de Área,Nome da Área,Ano,Período de Tempo,Propriedade,Nível NAICS,Código NAICS,Nome da Indústria,Estabelecimentos,Emprego Médio Mensal,Emprego no 1º Mês,Emprego no 2º Mês,Emprego no 3º Mês,Salários Totais (Todos os Trabalhadores),Salário Médio Semanal
202638,Condado,Condado de Alpine,2020,Anual,Setor Privado,2,44-45,Retail Trade,4,13,0,0,0,315141.0,485.0
202639,Condado,Condado de Butte,2020,Anual,Setor Privado,6,445210,Meat Markets,4,25,0,0,0,1210395.0,940.0
202640,Condado,Condado de Calaveras,2020,Anual,Setor Privado,3,221,Utilities,5,108,0,0,0,15775439.0,2820.0
202641,Condado,Condado de Calaveras,2020,Anual,Setor Privado,2,23,Construction,180,866,0,0,0,51419983.0,1142.0
202642,Condado,Condado de Alameda,2020,Anual,Setor Privado,6,334510,Electromedical Apparatus Manufacturing,37,1692,0,0,0,285925551.0,3250.0


### *-- Feature Engineering -> Adicionando Coluna de Setores*

In [16]:
# Importando dicionário de outro arquivo para captar setores e subsetores
from pegarSetores import pegar_setores


setores = pegar_setores()

def aplicar_setor(linha):

    valorCell = linha["Nome da Indústria"]

    if valorCell in setores:
        return setores[valorCell]
    else:
        return "NÃO LISTADO"

In [17]:
# Aplicando a função para ambas as tabelas

df["Setor"] = df.apply(aplicar_setor, axis=1)
df_anual["Setor"] = df_anual.apply(aplicar_setor, axis=1)

In [18]:
display(df.head())
print("\n-------------------\n")
display(df_anual.head())

,Tipo de Área,Nome da Área,Ano,Período de Tempo,Propriedade,Nível NAICS,Código NAICS,Nome da Indústria,Estabelecimentos,Emprego Médio Mensal,Emprego no 1º Mês,Emprego no 2º Mês,Emprego no 3º Mês,Salários Totais (Todos os Trabalhadores),Salário Médio Semanal,Setor
0,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,2,1024,Professional and Business Services,2,9,9,9,9,279730.0,2391.0,Generalistas
1,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,3,491,Postal Service,28,3189,3198,3181,3189,54692130.0,1319.0,Transportation and Warehousing
2,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,3,541,Professional and Technical Services,2,9,9,9,9,279730.0,2391.0,NÃO LISTADO
3,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,5,54133,Engineering Services,1,2,2,2,2,31831.0,1224.0,"Professional, Scientific, and Technical Services"
4,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,2,62,Health Care and Social Assistance,1,376,384,376,369,8804659.0,1800.0,Health Care and Social Assistance



-------------------



,Tipo de Área,Nome da Área,Ano,Período de Tempo,Propriedade,Nível NAICS,Código NAICS,Nome da Indústria,Estabelecimentos,Emprego Médio Mensal,Emprego no 1º Mês,Emprego no 2º Mês,Emprego no 3º Mês,Salários Totais (Todos os Trabalhadores),Salário Médio Semanal,Setor
202638,Condado,Condado de Alpine,2020,Anual,Setor Privado,2,44-45,Retail Trade,4,13,0,0,0,315141.0,485.0,Retail Trade
202639,Condado,Condado de Butte,2020,Anual,Setor Privado,6,445210,Meat Markets,4,25,0,0,0,1210395.0,940.0,NÃO LISTADO
202640,Condado,Condado de Calaveras,2020,Anual,Setor Privado,3,221,Utilities,5,108,0,0,0,15775439.0,2820.0,Utilities
202641,Condado,Condado de Calaveras,2020,Anual,Setor Privado,2,23,Construction,180,866,0,0,0,51419983.0,1142.0,Construction
202642,Condado,Condado de Alameda,2020,Anual,Setor Privado,6,334510,Electromedical Apparatus Manufacturing,37,1692,0,0,0,285925551.0,3250.0,NÃO LISTADO


#### *--- Adicionando Setores Ainda Não Listados*

In [19]:
# Classificando por Código

codigos_setor = {
    "11": "Agriculture, Forestry, Fishing and Hunting",
    "21": "Mining, Quarrying, and Oil and Gas Extraction",
    "22": "Utilities",
    "23": "Construction",
    # "31-33": "Manufacturing",
    "31": "Manufacturing",
    "32": "Manufacturing",
    "33": "Manufacturing",
    "42": "Wholesale Trade",
    # "44-45": "Retail Trade",
    "44": "Retail Trade",
    "45": "Retail Trade",
    # "48-49": "Transportation and Warehousing",
    "48": "Transportation and Warehousing",
    "49": "Transportation and Warehousing",
    "51": "Information",
    "52": "Finance and Insurance",
    "53": "Real Estate and Rental and Leasing",
    "54": "Professional, Scientific, and Technical Services",
    "55": "Management of Companies and Enterprises",
    "56": "Administrative and Support and Waste Management and Remediation Services",
    "61": "Educational Services",
    "62": "Health Care and Social Assistance",
    "71": "Arts, Entertainment, and Recreation",
    "72": "Accommodation and Food Services",
    "81": "Other Services (except Public Administration)",
    "92": "Public Administration (not covered in economic census)"
}


In [20]:
#Criando uma tabela temporária trimestral
temp_df = df[df["Setor"] == "NÃO LISTADO"].copy()
temp_df["Setor"] = temp_df["Código NAICS"].map(lambda cell: codigos_setor[cell.strip()[:2]] if cell.strip()[:2] in codigos_setor else "NÃO LISTADO")


#Criando uma tabela temporária anual
temp_df_anual = df_anual[df_anual["Setor"] == "NÃO LISTADO"].copy()
temp_df_anual["Setor"] = temp_df_anual["Código NAICS"].map(lambda cell: codigos_setor[cell.strip()[:2]] if cell.strip()[:2] in codigos_setor else "NÃO LISTADO")


display(temp_df.head())
print("\n-------------------\n")
display(temp_df_anual.head())

,Tipo de Área,Nome da Área,Ano,Período de Tempo,Propriedade,Nível NAICS,Código NAICS,Nome da Indústria,Estabelecimentos,Emprego Médio Mensal,Emprego no 1º Mês,Emprego no 2º Mês,Emprego no 3º Mês,Salários Totais (Todos os Trabalhadores),Salário Médio Semanal,Setor
2,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,3,541,Professional and Technical Services,2,9,9,9,9,279730.0,2391.0,"Professional, Scientific, and Technical Services"
10,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,6,522298,All Other Nondeposit Credit Intermediatn,1,11,11,12,12,369426.0,2436.0,Finance and Insurance
11,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,2,54,Professional and Technical Services,2,9,9,9,9,279730.0,2391.0,"Professional, Scientific, and Technical Services"
13,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,4,5416,Management & Technical Consulting Svc,1,7,7,7,7,247899.0,2724.0,"Professional, Scientific, and Technical Services"
15,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,3,621,Ambulatory Health Care Services,1,376,384,376,369,8804659.0,1800.0,Health Care and Social Assistance



-------------------



,Tipo de Área,Nome da Área,Ano,Período de Tempo,Propriedade,Nível NAICS,Código NAICS,Nome da Indústria,Estabelecimentos,Emprego Médio Mensal,Emprego no 1º Mês,Emprego no 2º Mês,Emprego no 3º Mês,Salários Totais (Todos os Trabalhadores),Salário Médio Semanal,Setor
202639,Condado,Condado de Butte,2020,Anual,Setor Privado,6,445210,Meat Markets,4,25,0,0,0,1210395.0,940.0,Retail Trade
202642,Condado,Condado de Alameda,2020,Anual,Setor Privado,6,334510,Electromedical Apparatus Manufacturing,37,1692,0,0,0,285925551.0,3250.0,Manufacturing
202643,Condado,Condado de Alameda,2020,Anual,Setor Privado,6,334511,"Search, Detection & Navigation Instrumnt",5,441,0,0,0,32993301.0,1440.0,Manufacturing
202644,Condado,Condado de Alameda,2020,Anual,Setor Privado,4,3351,Electric Lighting Equipment Mfg,18,647,0,0,0,61920153.0,1841.0,Manufacturing
202645,Condado,Condado de Alameda,2020,Anual,Setor Privado,5,33592,Communication and Energy Wire and Cable,9,421,0,0,0,45986055.0,2099.0,Manufacturing


In [21]:
df[df["Setor"] == "NÃO LISTADO"] = temp_df

df_anual[df_anual["Setor"] == "NÃO LISTADO"] = temp_df_anual

display(df.head())
print("\n-------------------\n")
display(df_anual.head())

,Tipo de Área,Nome da Área,Ano,Período de Tempo,Propriedade,Nível NAICS,Código NAICS,Nome da Indústria,Estabelecimentos,Emprego Médio Mensal,Emprego no 1º Mês,Emprego no 2º Mês,Emprego no 3º Mês,Salários Totais (Todos os Trabalhadores),Salário Médio Semanal,Setor
0,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,2,1024,Professional and Business Services,2,9,9,9,9,279730.0,2391.0,Generalistas
1,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,3,491,Postal Service,28,3189,3198,3181,3189,54692130.0,1319.0,Transportation and Warehousing
2,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,3,541,Professional and Technical Services,2,9,9,9,9,279730.0,2391.0,"Professional, Scientific, and Technical Services"
3,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,5,54133,Engineering Services,1,2,2,2,2,31831.0,1224.0,"Professional, Scientific, and Technical Services"
4,Condado,Condado de Alameda,2020,Trimestre 1,Governo Federal,2,62,Health Care and Social Assistance,1,376,384,376,369,8804659.0,1800.0,Health Care and Social Assistance



-------------------



,Tipo de Área,Nome da Área,Ano,Período de Tempo,Propriedade,Nível NAICS,Código NAICS,Nome da Indústria,Estabelecimentos,Emprego Médio Mensal,Emprego no 1º Mês,Emprego no 2º Mês,Emprego no 3º Mês,Salários Totais (Todos os Trabalhadores),Salário Médio Semanal,Setor
202638,Condado,Condado de Alpine,2020,Anual,Setor Privado,2,44-45,Retail Trade,4,13,0,0,0,315141.0,485.0,Retail Trade
202639,Condado,Condado de Butte,2020,Anual,Setor Privado,6,445210,Meat Markets,4,25,0,0,0,1210395.0,940.0,Retail Trade
202640,Condado,Condado de Calaveras,2020,Anual,Setor Privado,3,221,Utilities,5,108,0,0,0,15775439.0,2820.0,Utilities
202641,Condado,Condado de Calaveras,2020,Anual,Setor Privado,2,23,Construction,180,866,0,0,0,51419983.0,1142.0,Construction
202642,Condado,Condado de Alameda,2020,Anual,Setor Privado,6,334510,Electromedical Apparatus Manufacturing,37,1692,0,0,0,285925551.0,3250.0,Manufacturing


## *-- Traduzindo Setores e Subsetores*

In [22]:
from deep_translator import GoogleTranslator

tradutor = GoogleTranslator(source="en", target="pt")

In [23]:
# Traduzindo Subsetores

lista_subsetores = set(df["Nome da Indústria"])

tradutor = GoogleTranslator(source="en", target="pt")

for i, subsetor in enumerate(lista_subsetores):
    traduzido = tradutor.translate(subsetor)
    
    df.loc[df["Nome da Indústria"] == subsetor, "Nome da Indústria"] = traduzido
    df_anual.loc[df_anual["Nome da Indústria"] == subsetor, "Nome da Indústria"] = traduzido

    print(f"Linha {i + 1} traduzida")

Linha 1 traduzida
Linha 2 traduzida
Linha 3 traduzida
Linha 4 traduzida
Linha 5 traduzida
Linha 6 traduzida
Linha 7 traduzida
Linha 8 traduzida
Linha 9 traduzida
Linha 10 traduzida
Linha 11 traduzida
Linha 12 traduzida
Linha 13 traduzida
Linha 14 traduzida
Linha 15 traduzida
Linha 16 traduzida
Linha 17 traduzida
Linha 18 traduzida
Linha 19 traduzida
Linha 20 traduzida
Linha 21 traduzida
Linha 22 traduzida
Linha 23 traduzida
Linha 24 traduzida
Linha 25 traduzida
Linha 26 traduzida
Linha 27 traduzida
Linha 28 traduzida
Linha 29 traduzida
Linha 30 traduzida
Linha 31 traduzida
Linha 32 traduzida
Linha 33 traduzida
Linha 34 traduzida
Linha 35 traduzida
Linha 36 traduzida
Linha 37 traduzida
Linha 38 traduzida
Linha 39 traduzida
Linha 40 traduzida
Linha 41 traduzida
Linha 42 traduzida
Linha 43 traduzida
Linha 44 traduzida
Linha 45 traduzida
Linha 46 traduzida
Linha 47 traduzida
Linha 48 traduzida
Linha 49 traduzida
Linha 50 traduzida
Linha 51 traduzida
Linha 52 traduzida
Linha 53 traduzida
Li

In [24]:
# Traduzindo Setores

lista_setores = set(df["Setor"])

tradutor = GoogleTranslator(source="en", target="pt")

for i, setor in enumerate(lista_setores):
    traduzido = tradutor.translate(setor)

    df.loc[df["Setor"] == setor, "Setor"] = traduzido
    df_anual.loc[df_anual["Setor"] == setor, "Setor"] = traduzido

    print(f"Linha {i + 1} traduzida")

Linha 1 traduzida
Linha 2 traduzida
Linha 3 traduzida
Linha 4 traduzida
Linha 5 traduzida
Linha 6 traduzida
Linha 7 traduzida
Linha 8 traduzida
Linha 9 traduzida
Linha 10 traduzida
Linha 11 traduzida
Linha 12 traduzida
Linha 13 traduzida
Linha 14 traduzida
Linha 15 traduzida
Linha 16 traduzida
Linha 17 traduzida
Linha 18 traduzida
Linha 19 traduzida
Linha 20 traduzida
Linha 21 traduzida
Linha 22 traduzida
Linha 23 traduzida


## *-- Salvando Arquivos*

In [28]:
df.to_csv("Dados/Tratados/QCEW_Setorizado.csv", index=False, sep=";")
df_anual.to_csv("Dados/Tratados/QCEW_Anual_Setorizado.csv", index=False, sep=";")

# *- Loading na S3*

In [29]:
import os
import boto3
from dotenv import load_dotenv


# Carregar variáveis
load_dotenv(r"credentials/.env")

# Configure as cresdencias da AWS
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

# Crie um client S3
s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

# Carregue o arquivo trimestral para o S3
s3.upload_file(
    Filename='Dados/Tratados/QCEW_Setorizado.csv',
    Bucket='projetofinal-consuelo',
    Key='QCEW_Setorizado.csv'
)

# Carregue o arquivo anual para o S3
s3.upload_file(
    Filename='Dados/Tratados/QCEW_Anual_Setorizado.csv',
    Bucket='projetofinal-consuelo',
    Key='QCEW_Anual_Setorizado.csv'
)
